In [ ]:
!pip3 install mxnet
!pip3 install gluoncv

Image Classification

In [ ]:
import mxnet as mx
import gluoncv as gcv
import matplotlib.pyplot as plt


In [ ]:
image = mx.image.imread('guy.jpg')

In [ ]:
print('type:',type(image))

In [ ]:
print('shape:',image.shape)

In [ ]:
print('data type:',image.dtype)

In [ ]:
print('minimum value:', image.min().asscalar())
print('maximum value:',image.max().asscalar())

In [ ]:
plt.imshow(image.asnumpy())

In [ ]:
image = gcv.data.transforms.presets.imagenet.transform_eval(image)

In [ ]:
print('shape:',image.shape)
print('data type:',image.dtype)
print('minimum value:',image.min().asscalar())
print('maximum value:',image.max().asscalar())

load model


In [ ]:
network = gcv.model_zoo.resnet50_v1d(pretrained=True)

In [ ]:
prediction = network(image)

In [ ]:

print(prediction.shape)

In [ ]:

prediction = prediction[0]

In [ ]:
print(prediction.shape)

In [ ]:
print('Skipping 950 values...')
print(prediction[950:])

Calculate Probability

In [ ]:

probability = mx.nd.softmax(prediction)

In [ ]:
rounded_probability = mx.nd.round(probability*100)/100
print('Skipping 950 values..')
print(rounded_probability[950:])

In [ ]:

mx.nd.softmax(mx.nd.array((-0.5,0.5)))

Get top-k

In [ ]:
k = 5
topk_indicies = mx.nd.topk(probability, k=k)
topk_indicies

In [ ]:
network.classes[567]

In [ ]:
for i in range(k):
  class_index = topk_indicies[i].astype('int').asscalar()
  class_label = network.classes[class_index]
  class_probability = probability[class_index]
  print('#{} {} ({:0.3}%)'.format(i+1, class_label, class_probability.asscalar()*100))

In [ ]:
print('shape:',image.shape)
print('data type:',image.dtype)
print('minimum value:',image.min().asscalar())
print('maximum value:',image.max().asscalar())

Object Detection


In [ ]:
image = mx.image.imread('guy.jpg')

In [ ]:

image ,chw_image = gcv.data.transforms.presets.yolo.transform_test(image, short=512)
print('shape:',image.shape)
print('data type:',image.dtype)
print('minimum value:',image.min().asscalar())
print('maximum value:',image.max().asscalar())

In [ ]:
plt.imshow(chw_image)

In [ ]:

network = gcv.model_zoo.get_model('yolo3_darknet53_coco', pretrained=True)

In [ ]:

prediction = network(image)

In [ ]:
type(prediction)

In [ ]:
for index, array in enumerate(prediction):
  print('#{} shape:{}'.format(index + 1, array.shape))

In [ ]:
prediction = [array[0] for array in prediction]
class_indicies, probabilities, bounding_boxes = prediction

In [ ]:
k = 10
print(class_indicies[:k])

In [ ]:
network.classes

In [ ]:
print(probabilities[:k])

In [ ]:
print(bounding_boxes[:k])

In [ ]:
gcv.utils.viz.plot_bbox(chw_image,bounding_boxes,probabilities,class_indicies, class_names = network.classes)

Image segmentation

In [ ]:
image = mx.image.imread('guy.jpg')
print('shape:',image.shape)
print('data type:',image.dtype)
print('minimum value:',image.min().asscalar())
print('maximum value:',image.max().asscalar())

In [ ]:
plt.imshow(image.asnumpy())

In [ ]:

from mxnet.gluon.data.vision import transforms
transform_fn = transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize([.485,.456,.406],[.229,.224,.225])
])

In [ ]:
image = transform_fn(image)
print('shape:',image.shape)
print('data type:',image.dtype)
print('minimum value:',image.min().asscalar())
print('maximum value:', image.max().asscalar())

In [ ]:
image = image.expand_dims(0)
print(image.shape)

In [ ]:
network = gcv.model_zoo.get_model('deeplab_resnet50_ade',pretrained=True)

In [ ]:
network = gcv.model_zoo.get_model('fcn_resnet50_ade',pretrained=True)

In [ ]:
output = network.demo(image)
print(output.shape)

In [ ]:
output = output[0]
print(output.shape)

pixel slice


In [ ]:
px_height, px_width = 300, 500
px_logit = output[:,px_height, px_width]

In [ ]:
px_probability = mx.nd.softmax(px_logit)
px_rounded_probability = mx.nd.round(px_probability*100)/100
print(px_rounded_probability)

In [ ]:
prediction = mx.nd.argmax(output,0).asnumpy()
print(prediction.shape)

In [ ]:

print(prediction)

In [ ]:
from gluoncv.utils.viz import get_color_pallete
prediction_image = get_color_pallete(prediction, 'ade20k')
plt.imshow(prediction_image)

In [ ]:
from PIL import Image, ImageEnhance
im = Image.open("guy.jpg")
enhancer = ImageEnhance.Sharpness(im)
enhanced_im = enhancer.enhance(10.0)
enhanced_im.save("enhanced.png")

In [ ]:
from PIL import Image, ImageEnhance
im = Image.open("guy.jpg")
enhancer = ImageEnhance.Brightness(im)
enhanced_im = enhancer.enhance(1.8)
enhanced_im.save("enhanced-brightness.png")

In [ ]:
from PIL import Image, ImageEnhance
im = Image.open("guy.jpg")
enhancer = ImageEnhance.Contrast(im)
enhanced_im = enhancer.enhance(4.0)
enhanced_im.save("enhanced-contrast.png")